# [프로젝트2] AlexNet을 활용한 이미지 분류 모델 학습 및 추론


---


## 실습 목표
---
- **목표 : AlexNet을 활용하여 이미지 분류 수행**
  - AlexNet을 활용하여 딥러닝 기반 이미지 분류를 수행하고, 그 성능을 확인합니다.


## 실습 목차
---

1. **학습 준비**: 딥러닝 모델을 구현하기 위한 라이브러리를 호출하고, 모델을 정의합니다. 

2. **데이터셋 및 데이터 로더 준비**: 학습에 필요한 데이터를 불러오고 전처리 합니다

3. **손실 함수와 최적화 알고리즘 설정**: 모델의 학습을 위한 손실 함수와 최적화 함수(optimizer)를 정의합니다

4. **학습 실행**: 앞서 정의한 데이터와 손실 함수, optimizer를 활용해 모델을 학습합니다.

5. **모델 성능 평가**: 학습이 완료된 모델의 분류 성능을 평가합니다.

## 1. 학습 준비
---
필요 라이브러리를 호출합니다.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

GPU를 사용할 수 있는지 확인합니다. GPU를 사용할 수 있다면 device에 cuda를, 사용할 수 없다면 cpu를 할당합니다.

In [2]:
# GPU 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#### [TODO] AlexNet 모델을 정의해주세요.
- AlexNet 모델을 정의하고, 가능하다면 모델을 GPU에 이동합니다. 그리고 모델의 구조를 출력합니다. 

In [4]:
# AlexNet 모델 정의
model = models.alexnet(pretrained=False)
# 최종 output layer 개수 1000개로 기본 설정되어 있으나, 출력 클래스 2개 할 것이므로 False
print(model) # 원래 out_features=1000
print("******************************************************************")

num_features = model.classifier[6].in_features # Classifier의 가장 마지막 layer의 input 개수
model.classifier[6] = nn.Linear(num_features, 2)  # 출력 클래스 개수: 2 (tower, non-tower)

# 가능하다면 GPU 사용
model = model.to(device)

# 모델의 구조 출력
print(model)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

## 2. 데이터셋 및 데이터 로더 준비
---
#### [TODO] 데이터 전처리를 위한 변환 과정을 정의해주세요. 
- 이미지 크기를 224, 224 로 조정하는 레이어를 추가해보세요.

In [5]:
# 데이터 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 이미지 크기 조정
    transforms.ToTensor(),           # 이미지를 Tensor로 변환
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  # 이미지 정규화 / 0~1 사이의 픽셀값을 해당값들의 평균과 표준편차로 변환하여 학습 진행
])

#### [TODO] 데이터셋을 불러오고, 데이터 로더를 생성하는 코드를 작성해주세요.

In [6]:
# 데이터 경로 설정
data_path = "/mnt/elice/dataset/classification_data/"

train_path = data_path + "train/"
test_path = data_path + "test/"

In [7]:
# 데이터셋 로딩
train_dataset = datasets.ImageFolder(train_path, transform=transform)
test_dataset = datasets.ImageFolder(test_path, transform=transform)

# 데이터로더 생성
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False)

## 3. 손실 함수와 최적화 알고리즘 설정
---
학습을 위한 손실 함수와 최적화 알고리즘 (optimizer)을 정의합니다.

In [9]:
criterion = nn.CrossEntropyLoss()
# CrossEntropyLoss : 각 클래스의 확률값이 한 쪽 카테고리 값으로 크게 나타나면 loss 작아짐
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
# momentum : 현재 업데이트되는 weight 값에 이전 단계 weight를 0.9 반영하여 업데이트

## 4. 학습 실행
---
10 epoch 만큼 학습을 진행합니다.
- 학습에는 약 5분 정도 소요됩니다.

In [10]:
import time
start_time = time.time()

# 학습
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)

    train_loss /= len(train_loader.dataset)
    # 이미지당 평균 loss 값


    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}")

end_time = time.time()

training_time = end_time - start_time

print(f"학습 시간: {training_time} 초\n")

Epoch 1/10, Train Loss: 0.6902
Epoch 2/10, Train Loss: 0.6787
Epoch 3/10, Train Loss: 0.6673
Epoch 4/10, Train Loss: 0.6543
Epoch 5/10, Train Loss: 0.6433
Epoch 6/10, Train Loss: 0.6266
Epoch 7/10, Train Loss: 0.6086
Epoch 8/10, Train Loss: 0.5727
Epoch 9/10, Train Loss: 0.5330
Epoch 10/10, Train Loss: 0.4968
학습 시간: 39.10748291015625 초



학습 완료한 모델을 저장합니다.

In [11]:
# 모델 저장
filename = f"model_alexnet_sample1.pt"
torch.save(model, filename)

## 5. 모델 성능 평가
---
테스트 데이터에 대해 모델의 성능을 평가합니다. 평가 지표로는 Accuracy를 사용합니다. 

#### [TODO] 모델을 검증 모드로 변경해주세요.

In [12]:
# 모델을 검증 모드로 변환합니다
model.eval()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [13]:
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1) # 1 열 단위로
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")

Test Accuracy: 73.42%
